In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.1
seed = 0
alpha = 1e-2
lamda = 1e-3
eta = 1e-2
d0 = 8
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.1        0.11111111 0.11111111 0.11111111 0.
  0.11111111 0.         0.11111111 0.         0.         0.
  0.         0.         0.         0.         0.09090909 0.1
  0.         0.        ]
 [0.1        0.         0.         0.1        0.1        0.
  0.         0.         0.1        0.1        0.         0.1
  0.         0.         0.         0.1        0.09090909 0.
  0.         0.1       ]
 [0.11111111 0.         0.         0.         0.         0.14285714
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.09090909 0.
  0.2        0.        ]
 [0.11111111 0.1        0.         0.         0.         0.
  0.14285714 0.         0.11111111 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.11111111 0.1        0.         0.         0.         0.
  0.         0.         0.11111111 0.         0.         0.
  0.         0.         0.         0.         0.09

In [4]:
def load_dataset():
    transforms_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    mnist_data_train = datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms_mnist)
    mnist_data_test = datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms_mnist)

    return mnist_data_train, mnist_data_test

In [5]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0, 0.0, 5.0, -2.0, -3.0, 5.0, 2.0])
W2 = np.array([2.0, 2.0, 0.0, 5.0, 2.0, 3.0, -5.0, -2.0])
W = [W1, W2]
m = 200
n = 8
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(8, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)

        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss.detach(), parameters_to_vector(model.parameters()).detach())
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

0 tensor(87.6804) tensor([ 0.3305, -0.2253,  0.1037, -0.2817,  0.0087,  0.1417, -0.1966,  0.3161])
0 tensor(60.9681) tensor([ 0.3962, -0.1782,  0.1119, -0.1343,  0.0459,  0.1601, -0.3044,  0.2658])
0 tensor(53.2318) tensor([ 0.3998, -0.1879,  0.1252, -0.0436,  0.0746,  0.2277, -0.4025,  0.2383])
0 tensor(76.6574) tensor([ 0.4287, -0.1146,  0.0801,  0.0010,  0.1190,  0.2734, -0.4774,  0.2068])
1 tensor(74.8370) tensor([ 0.4473, -0.0747,  0.0627,  0.1299,  0.1462,  0.3161, -0.5867,  0.1562])
1 tensor(52.1197) tensor([ 0.5082, -0.0305,  0.0709,  0.2658,  0.1809,  0.3339, -0.6858,  0.1099])
1 tensor(45.5295) tensor([ 0.5117, -0.0389,  0.0836,  0.3491,  0.2076,  0.3974, -0.7757,  0.0839])
1 tensor(65.3973) tensor([ 0.5385,  0.0293,  0.0421,  0.3901,  0.2487,  0.4402, -0.8447,  0.0544])
2 tensor(63.8939) tensor([ 0.5559,  0.0668,  0.0266,  0.5086,  0.2738,  0.4806, -0.9452,  0.0075])
2 tensor(44.5751) tensor([ 0.6123,  0.1082,  0.0347,  0.6339,  0.3061,  0.4978, -1.0363, -0.0351])
2 tensor(3

23 tensor(2.4655) tensor([ 1.6867,  1.5968, -0.1549,  4.1494,  1.5913,  2.3566, -4.3179, -1.5474])
23 tensor(1.8395) tensor([ 1.6981,  1.6061, -0.1505,  4.1734,  1.5989,  2.3625, -4.3332, -1.5553])
23 tensor(1.4950) tensor([ 1.6998,  1.6055, -0.1457,  4.1872,  1.6053,  2.3778, -4.3465, -1.5623])
23 tensor(2.1224) tensor([ 1.7049,  1.6187, -0.1515,  4.1937,  1.6125,  2.3867, -4.3576, -1.5695])
24 tensor(2.1181) tensor([ 1.7088,  1.6268, -0.1518,  4.2131,  1.6174,  2.3976, -4.3736, -1.5782])
24 tensor(1.5876) tensor([ 1.7194,  1.6355, -0.1476,  4.2353,  1.6244,  2.4032, -4.3877, -1.5854])
24 tensor(1.2818) tensor([ 1.7210,  1.6349, -0.1430,  4.2481,  1.6305,  2.4175, -4.3999, -1.5921])
24 tensor(1.8218) tensor([ 1.7257,  1.6472, -0.1483,  4.2541,  1.6371,  2.4257, -4.4101, -1.5988])
25 tensor(1.8202) tensor([ 1.7293,  1.6547, -0.1484,  4.2720,  1.6417,  2.4360, -4.4248, -1.6068])
25 tensor(1.3707) tensor([ 1.7391,  1.6627, -0.1444,  4.2925,  1.6483,  2.4413, -4.4377, -1.6136])
25 tensor(

50 tensor(0.0398) tensor([ 1.9587,  1.9538, -0.0556,  4.8956,  1.9271,  2.8925, -4.9348, -1.9322])
50 tensor(0.0254) tensor([ 1.9590,  1.9536, -0.0544,  4.8973,  1.9284,  2.8951, -4.9360, -1.9335])
50 tensor(0.0382) tensor([ 1.9596,  1.9553, -0.0547,  4.8980,  1.9293,  2.8963, -4.9372, -1.9348])
51 tensor(0.0397) tensor([ 1.9602,  1.9563, -0.0542,  4.9003,  1.9302,  2.8983, -4.9389, -1.9359])
51 tensor(0.0347) tensor([ 1.9617,  1.9574, -0.0530,  4.9032,  1.9314,  2.8994, -4.9402, -1.9367])
51 tensor(0.0219) tensor([ 1.9620,  1.9572, -0.0519,  4.9048,  1.9326,  2.9018, -4.9414, -1.9379])
51 tensor(0.0331) tensor([ 1.9625,  1.9587, -0.0522,  4.9054,  1.9335,  2.9029, -4.9425, -1.9391])
52 tensor(0.0344) tensor([ 1.9631,  1.9597, -0.0517,  4.9075,  1.9343,  2.9047, -4.9440, -1.9401])
52 tensor(0.0303) tensor([ 1.9644,  1.9607, -0.0506,  4.9102,  1.9354,  2.9058, -4.9452, -1.9410])
52 tensor(0.0189) tensor([ 1.9647,  1.9606, -0.0495,  4.9117,  1.9366,  2.9081, -4.9463, -1.9421])
52 tensor(

76 tensor(0.0013) tensor([ 1.9942,  1.9946, -0.0147,  4.9849,  1.9844,  2.9803, -4.9938, -1.9885])
76 tensor(0.0006) tensor([ 1.9942,  1.9945, -0.0144,  4.9852,  1.9847,  2.9808, -4.9939, -1.9887])
76 tensor(0.0010) tensor([ 1.9943,  1.9947, -0.0144,  4.9853,  1.9849,  2.9809, -4.9940, -1.9890])
77 tensor(0.0011) tensor([ 1.9944,  1.9949, -0.0142,  4.9855,  1.9850,  2.9813, -4.9942, -1.9891])
77 tensor(0.0011) tensor([ 1.9946,  1.9950, -0.0139,  4.9860,  1.9853,  2.9815, -4.9943, -1.9892])
77 tensor(0.0005) tensor([ 1.9947,  1.9950, -0.0137,  4.9862,  1.9856,  2.9820, -4.9944, -1.9895])
77 tensor(0.0009) tensor([ 1.9947,  1.9952, -0.0136,  4.9863,  1.9857,  2.9821, -4.9946, -1.9897])
78 tensor(0.0010) tensor([ 1.9948,  1.9953, -0.0135,  4.9866,  1.9859,  2.9825, -4.9947, -1.9898])
78 tensor(0.0010) tensor([ 1.9950,  1.9954, -0.0132,  4.9870,  1.9861,  2.9827, -4.9948, -1.9899])
78 tensor(0.0005) tensor([ 1.9951,  1.9954, -0.0130,  4.9872,  1.9864,  2.9831, -4.9949, -1.9901])
78 tensor(

In [11]:
parameters_to_vector(model.parameters())

tensor([ 1.9991e+00,  1.9993e+00, -3.9728e-03,  4.9973e+00,  1.9960e+00,
         2.9957e+00, -4.9995e+00, -1.9976e+00], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, models, first_run=True):
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        
                        # Generate random values from a normal distribution for the diagonal
                        diag_values = 1.0 + 1.0 * torch.randn(row)

                        # Create a diagonal matrix with random values
                        mat = torch.diag(diag_values)
                        
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights, models)



In [15]:
print(projection_list[0])

[0, 0, tensor([[0.3029, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4255, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4052, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3678, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 2.0692, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8769, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1363, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.3007]]), 0, 0, tensor([[-0.4598,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0492,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.9550,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.8257,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.2603,  0.0000,  0

In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [18]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(8.6603, dtype=torch.float64)

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([ 0.0147,  0.1417,  0.1821, -0.2452, -0.0107, -0.2535, -0.0555, -0.2902])


tensor([ 0.0293,  0.2833,  0.3643, -0.4904, -0.0213, -0.5071, -0.1111, -0.5805],
       grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

in_cluster_proj_norm = []
out_cluster_proj_norm = []
in_cluster_proj_diff_norm = []
out_cluster_proj_diff_norm = []


for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    projected_weights = []
    update_ProjWeight(projection_list, projected_weights, models, first_run=False)
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            temp_mat = torch.outer(projected_weights[i][j] - projected_weights[j][i], weights).clone()
            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
    
    in_cluster_proj_norm_round = 0
    out_cluster_proj_norm_round = 0
    in_cluster_proj_diff_round = 0
    out_cluster_proj_diff_round = 0
    in_edges = 0
    out_edges = 0
    
    for i in range(no_users//2):
        for j in G.neighbors(i):
            if j < no_users//2:
                in_edges += 1
                in_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                in_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
            else:
                out_edges += 1
                out_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                out_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
    in_cluster_proj_norm.append(in_cluster_proj_norm_round / in_edges)
    out_cluster_proj_norm.append(out_cluster_proj_norm_round / out_edges)
    in_cluster_proj_diff_norm.append(in_cluster_proj_diff_round / in_edges)
    out_cluster_proj_diff_norm.append(out_cluster_proj_diff_round / out_edges)
            
                                         
                                              
    
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 1/2000 [00:00<08:09,  4.08it/s]

Training_loss 61.17377,   Relative Error 0.95321


  0%|          | 2/2000 [00:00<07:39,  4.35it/s]

Training_loss 59.32458,   Relative Error 0.93878


  0%|          | 3/2000 [00:00<07:32,  4.42it/s]

...

Training_loss 0.00001,   Relative Error 0.00036
Training_loss 0.00001,   Relative Error 0.00036


100%|█████████▉| 1998/2000 [05:56<00:00,  6.51it/s]

Training_loss 0.00001,   Relative Error 0.00035
Training_loss 0.00001,   Relative Error 0.00035


100%|██████████| 2000/2000 [05:56<00:00,  5.61it/s]

Training_loss 0.00001,   Relative Error 0.00035
Training_loss 0.00001,   Relative Error 0.00035


In [21]:
#plot.plot(test_loss)
print(parameters_to_vector(models[19].parameters()), W[1])

tensor([ 2.0000e+00,  2.0002e+00,  1.3767e-04,  5.0002e+00,  2.0001e+00,
         3.0005e+00, -4.9999e+00, -1.9999e+00], grad_fn=<CatBackward0>) [ 2.  2.  0.  5.  2.  3. -5. -2.]


In [22]:
for i in G.neighbors(3):
    print(i)

1
6
17
19


In [24]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

in_cluster_proj_norm = np.array(in_cluster_proj_norm)
out_cluster_proj_norm = np.array(out_cluster_proj_norm)
in_cluster_proj_diff_norm = np.array(in_cluster_proj_diff_norm)
out_cluster_proj_diff_norm = np.array(out_cluster_proj_diff_norm)


In [25]:
np.save( 'training_loss_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)
np.save( 'in_cluster_proj_norm_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_norm)
np.save('out_cluster_proj_norm_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_norm)
np.save( 'in_cluster_proj_diff_norm_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_diff_norm)
np.save('out_cluster_proj_diff_norm_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_diff_norm)

In [26]:
'training_loss_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'training_loss_sheave_fml0_01_pout0_1'

In [27]:
'relative_error_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'relative_error_sheave_fml0_01_pout0_1'

In [28]:
'in_cluster_proj_norm_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'in_cluster_proj_norm_sheave_fml0_01_pout0_1'

In [29]:
'out_cluster_proj_norm_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'out_cluster_proj_norm_sheave_fml0_01_pout0_1'

In [30]:
'in_cluster_proj_diff_norm_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'in_cluster_proj_diff_norm_sheave_fml0_01_pout0_1'

In [31]:
'out_cluster_proj_diff_norm_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'out_cluster_proj_diff_norm_sheave_fml0_01_pout0_1'